In [1]:
import pandas as pd

data = pd.read_csv("groceriesSynthetic.tsv", sep = "\t")
df = pd.DataFrame(data)
df_original = pd.DataFrame(data)

In [2]:
#Checking if all the values in currency are EUR
print(df["currency"].nunique() == "EUR")

False


In [3]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo   value currency  
804    292384******9276 either JARVE SELVERI ISETEENI...  -10.74      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new    -5.7      SEK  
1806   (..6464) always sign protect everybody improve...  -22.44      SEK  
2530   (..5274) scene p

In [4]:
pip install CurrencyConverter

Note: you may need to restart the kernel to use updated packages.


In [5]:
df.head()

,id,formattedDate,merchantName,remittanceInfo,value,currency
0,1269,24-07-2023,NaN,567176******6366 toward information MAKSIMARKE...,-13.59,EUR
1,319,14-06-2023,NaN,157466******5153 pay LIDL own TARTU sound behi...,-0.89,EUR
2,981,28-10-2022,NaN,(..2754) politics garden property POHJAKESKUS ...,-1.68,EUR
3,1340,20-02-2023,MAKSIMARK small Port Kendrabury,24/04/1997 resource kaart...878232 VILJANDI MA...,-12.9,EUR
4,604,15-05-2023,OU rather Lake Jorgebury,14/05/1977 such kaart...983921 less OUe/Tallin...,-37.32,EUR


In [6]:
from currency_converter import CurrencyConverter

c = CurrencyConverter()

def currency_to_EUR(value, currency):
    if currency != "EUR":
        try:
            conversion = c.convert(value, currency, "EUR")
            return round(conversion, 2)
        except Exception as e:
            print(f"Error converting {value} {currency} to EUR: {e}")
    return value

df["value"] = df.apply(lambda row: currency_to_EUR(row["value"], row["currency"]), axis=1)

In [7]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo  value currency  
804    292384******9276 either JARVE SELVERI ISETEENI... -10.06      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new  -0.49      SEK  
1806   (..6464) always sign protect everybody improve...  -1.93      SEK  
2530   (..5274) scene prepa

In [8]:
df = df.drop(columns="currency")

In [9]:
def findMerchant(x):
    if isinstance(x, str):
        mer = ""
        li = x.split()
        i = 0
        while li[i].isupper():
            mer + " " + li[i]
        return mer

In [10]:
#finding all the merchants in the merchantName column
#merchantsFromDf = df["merchantName"].apply(lambda x: findMerchant(x)).unique().tolist()
merchantsFromDf = df["merchantName"].apply(lambda x: x.split()[0] if isinstance(x, str) and x.split()[0].isupper() else None).dropna().unique().tolist()
print(merchantsFromDf)

print("-----------------------------")

import re
merchantsSet = set()
for index, row in df.iterrows():
    if pd.notnull(row["remittanceInfo"]) and isinstance(row["remittanceInfo"], str) and len(row["remittanceInfo"]) >= 1:
        listina = re.split(r'[ ,/]', row["remittanceInfo"])
        if len(listina) >= 3:
            if listina[2].isupper() and len(listina[2]) > 2:
                merchantsSet.add(listina[2])
                
merchantsFromDf2 = list(merchantsSet)
print(merchantsFromDf2)

['MAKSIMARK', 'OU', 'RIMI', 'SELVER', 'KAUPLUS', 'KAUBAMAJA', 'KONSUM', 'AS', 'MAXIMA', 'VALD', 'POOD', 'COOP', 'TOIDUKAUBAD', 'TALLINN', 'R-KIOSK', 'PRISMA', 'ISETEEN', 'OÜ', 'A', 'STOCKMANN', 'LIDL', 'KESKUS', 'GROSSI', 'TURU', 'ICA', 'TURG', 'ISETEENINDUS', 'MAKSEKESKUS', 'PAIDE', 'MEIE', 'KURESSAARE', 'MARKET', 'TARTU', 'WWW.RIMI.EE', 'A&O', 'R', 'VIRU', 'KARDLA', 'VILJANDI', 'PARNU', 'NARVA', 'KRISTIINE', 'JOHVI', 'BOLT', 'NARVESEN', 'KOHTLA-JARVE', 'TOIT', 'POLVA', 'MAARDU', 'SIKUPILLI', 'ROCCA', 'EEDEN', 'PUHVET', 'GET', 'BLOOM', 'APOLLO']
-----------------------------
['COOP', 'R-KIOSK', 'GEMOSSTARTU', 'TALUTURG', 'MESILASE', 'KRISTI', 'ASTE', '-OUCD27UAKJU48J4', 'TOIDUKAUBAD', 'A1000MARKET', 'NONATALLINNA', 'AASA', 'GOURMETCOFFEEKADRIORG', 'TURU', 'ALA', 'VILJANDI', 'EDUKESKUSE', 'KASSISABAKOIDU', 'ZOOKAUPLUS', 'EUROMARKETTALLINNA', 'KASEPERE', 'TARTU', 'CHEBURASHKA', 'LEIVAKAS', 'LUKUPOOD', 'SAARE', 'TASO', 'CARREFOURMARKET', 'VASVAL', 'S-MARKET', 'LEMMIKUKAUPLUSPARNU', 'ASTR

In [11]:
# Võtsin ära praegu merchantide listist mõned elemendid, mille pärast tuli pärastisel filtreerimisel
# mitu korda sama rida kuna nad sisaldasid mitut "merchantit" selle listi põhjal

elementsToRemove = ["OU", "AS", "VALD", "OÜ", "A", "R"]

for el in elementsToRemove:
    if el in merchantsFromDf:
        merchantsFromDf.remove(el)
    if el in merchantsFromDf2:
        merchantsFromDf2.remove(el)

In [12]:
merchantsFromDf

['MAKSIMARK',
 'RIMI',
 'SELVER',
 'KAUPLUS',
 'KAUBAMAJA',
 'KONSUM',
 'MAXIMA',
 'POOD',
 'COOP',
 'TOIDUKAUBAD',
 'TALLINN',
 'R-KIOSK',
 'PRISMA',
 'ISETEEN',
 'STOCKMANN',
 'LIDL',
 'KESKUS',
 'GROSSI',
 'TURU',
 'ICA',
 'TURG',
 'ISETEENINDUS',
 'MAKSEKESKUS',
 'PAIDE',
 'MEIE',
 'KURESSAARE',
 'MARKET',
 'TARTU',
 'WWW.RIMI.EE',
 'A&O',
 'VIRU',
 'KARDLA',
 'VILJANDI',
 'PARNU',
 'NARVA',
 'KRISTIINE',
 'JOHVI',
 'BOLT',
 'NARVESEN',
 'KOHTLA-JARVE',
 'TOIT',
 'POLVA',
 'MAARDU',
 'SIKUPILLI',
 'ROCCA',
 'EEDEN',
 'PUHVET',
 'GET',
 'BLOOM',
 'APOLLO']

In [13]:
#sellel on palju laiem valik poode
merchantsFromDf2
#oleks vaja leida viis kuidas eemaldada asulanimed
#web scrapeda linnanimed ja eemaldada need listist?

['COOP',
 'R-KIOSK',
 'GEMOSSTARTU',
 'TALUTURG',
 'MESILASE',
 'KRISTI',
 'ASTE',
 '-OUCD27UAKJU48J4',
 'TOIDUKAUBAD',
 'A1000MARKET',
 'NONATALLINNA',
 'AASA',
 'GOURMETCOFFEEKADRIORG',
 'TURU',
 'ALA',
 'VILJANDI',
 'EDUKESKUSE',
 'KASSISABAKOIDU',
 'ZOOKAUPLUS',
 'EUROMARKETTALLINNA',
 'KASEPERE',
 'TARTU',
 'CHEBURASHKA',
 'LEIVAKAS',
 'LUKUPOOD',
 'SAARE',
 'TASO',
 'CARREFOURMARKET',
 'VASVAL',
 'S-MARKET',
 'LEMMIKUKAUPLUSPARNU',
 'ASTRA',
 'PRISMA',
 'LIISASTRID',
 'SOULMARKKASTANI',
 'TALLINNA',
 'VASULA',
 'SUPERSA',
 'VALDEKU',
 'LOUNAKESKUSE',
 'LASNAMAE',
 'KASPAR',
 'KRISTIINE',
 'ESKU',
 'SUPERMARKET',
 'CASH',
 'REALISEERIMISKESKUSJAAMA',
 'LOO',
 'KAUNAS',
 'JONASE',
 'TABASALU',
 'PAASKULA',
 'KOOGIPOOD',
 'KUIVASTU',
 'KATHIKAS',
 'KASSARI',
 'LILLEKAUPLUS',
 'MAARAHVAPOOD',
 'KASMU',
 'FRESH',
 'MINIMARKET',
 'AASMAE',
 'VANA-PARNU',
 'COLOUR',
 '-ZP0MOUR4PKU4YZQ',
 'PURTSEGASTRO',
 'KONSUMPARNU',
 'KASSISABA',
 'EASYSMOKE',
 'LIHAPOOD',
 'JOUMEES',
 'MEGASTARSADAM

In [14]:
# Isegi pärast eelmise cell'i muudatusi on alles mõned duplicateid. Hetkel ei tea kuidas veel paremini neid filtreerida.

def findMerchant(row, merchantList):
    remittance_info = str(row["remittanceInfo"]).upper()
    for merchant in merchantList:
        if merchant in remittance_info:
            row["merchantName"] = merchant
            print(row.tolist())

test_df = df.copy()

counter = 0

for idx, row in test_df.iterrows():
    if pd.notnull(row["merchantName"]):
        findMerchant(row, merchantsFromDf)
        counter += 1
    if counter == 20:
        break

[1340, '20-02-2023', 'MAKSIMARK', '24/04/1997 resource kaart...878232 VILJANDI MAKSIMARKET/VILJANDIMAA,/EST carry', '-12.9']
[1340, '20-02-2023', 'MARKET', '24/04/1997 resource kaart...878232 VILJANDI MAKSIMARKET/VILJANDIMAA,/EST carry', '-12.9']
[1340, '20-02-2023', 'VILJANDI', '24/04/1997 resource kaart...878232 VILJANDI MAKSIMARKET/VILJANDIMAA,/EST carry', '-12.9']
[604, '15-05-2023', 'TALLINN', '14/05/1977 such kaart...983921 less OUe/Tallinn/EST animal', '-37.32']
[184, '07-09-2023', 'RIMI', '576266******5911 return RIMI/TATARI RIMI ISETE travel TALLINN', '-4.28']
[184, '07-09-2023', 'TALLINN', '576266******5911 return RIMI/TATARI RIMI ISETE travel TALLINN', '-4.28']
[535, '09-05-2023', 'SELVER', '29/05/1985 traditional kaart...609240 huge SELVERI ISETEENI/TALLINN/EST team', '-21.86']
[535, '09-05-2023', 'TALLINN', '29/05/1985 traditional kaart...609240 huge SELVERI ISETEENI/TALLINN/EST team', '-21.86']
[535, '09-05-2023', 'ISETEEN', '29/05/1985 traditional kaart...609240 huge SEL